# 第3章 大規模言語モデルの基礎

## 3.2 GPT（デコーダ）

### 3.2.4 transformersで使う

In [ ]:
!pip install transformers[ja,sentencepiece,torch] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.9/599.9 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 59.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import pipeline

In [ ]:
# 後続するテキストを予測するpipelineを作成
generator = pipeline(
    "text-generation", model="abeja/gpt2-large-japanese"
)
# "日本で一番高い山は"に続くテキストを生成
outputs = generator("日本で一番高い山は")
print(outputs[0]["generated_text"])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (50) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


日本で一番高い山は富士山ですが、富士山から一番近いのは琵琶湖ではなく東京なんだそうですよ。 では、湖は何度来ても東京のように見えるのかというとどうでしょうか。 琵琶湖は滋賀県にある、日本の湖だよ


## 3.3 BERT・RoBERTa（エンコーダ）

### 3.3.4 transformersで使う

In [ ]:
import pandas as pd

# マスクされたトークンを予測するpipelineを作成
fill_mask = pipeline(
    "fill-mask", model="cl-tohoku/bert-base-japanese-v3"
)
masked_text = "日本の首都は[MASK]である"
# [MASK]部分を予測
outputs = fill_mask(masked_text)
# 上位3件をテーブルで表示
display(pd.DataFrame(outputs[:3]))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v3 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,score,token,token_str,sequence
0,0.884170,12569,東京,日本 の 首都 は 東京 で ある
1,0.024820,12759,大阪,日本 の 首都 は 大阪 で ある
2,0.020864,13017,京都,日本 の 首都 は 京都 で ある


In [ ]:
masked_text = "今日の映画は刺激的で面白かった。この映画は[MASK]。"
outputs = fill_mask(masked_text)
display(pd.DataFrame(outputs[:3]))

,score,token,token_str,sequence
0,0.683933,23845,素晴らしい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 素晴らしい 。
1,0.101234,24683,面白い,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 面白い 。
2,0.048003,26840,楽しい,今日 の 映画 は 刺激 的 で 面白かっ た 。 この 映画 は 楽しい 。


## 3.4 T5（エンコーダ・デコーダ）

### 3.4.4 transformersで使う

In [ ]:
# text-to-textで生成するpipelineを作成
t2t_generator = pipeline(
    "text2text-generation", model="retrieva-jp/t5-large-long"
)
# マスクされたスパンを予測
masked_text = "江戸幕府を開いたのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

徳川家康


In [ ]:
t2t_generator.tokenizer.convert_tokens_to_ids("<extra_id_1>")

32098

In [ ]:
masked_text = "日本で通貨を発行しているのは、<extra_id_0>である"
outputs = t2t_generator(masked_text, eos_token_id=32098)
print(outputs[0]["generated_text"])

日本銀行


In [ ]:
"日本銀行" in t2t_generator.tokenizer.vocab

False